In [100]:
import pandas as pd
import numpy as np
import keras
from keras import models
from keras import layers
keras.__version__

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

'2.3.1'

In [106]:
from random import sample

In [76]:
df = pd.read_csv("000020.csv",parse_dates=['date'],index_col=[0])
df

,date,open,high,low,close,volume,pattern1,pattern2,pattern3,nasdaq
0,2012-01-02,4070,4100,4060,4085,25451,K00,NaN,NaN,NaN
1,2012-01-03,4100,4160,4100,4130,45291,P03,K00P03,NaN,NaN
2,2012-01-04,4135,4160,4125,4150,42911,K00,P00K00,P10K00,U04
3,2012-01-05,4150,4290,4110,4270,85140,P05,K00P05,P10P05,T01
4,2012-01-06,4300,4325,4220,4240,45535,M05,P00M05,P10M05,U04
...,...,...,...,...,...,...,...,...,...,...
2170,2020-10-28,17250,19400,17250,18950,757160,P03,P00P03,S00P03,U04
2171,2020-10-29,18400,19300,18300,19150,424896,P05,P00P05,P10P05,D04
2172,2020-10-30,19500,19500,17700,17900,459297,M01,P00M01,P10M01,U04
2173,2020-11-02,17850,18600,17650,18150,297779,P05,M00P05,S07P05,D04


In [77]:
df_res = pd.DataFrame(columns=['date','ce','cd','ct','he','ht','te','tt']) # 날짜 / 봉 유무 / 봉 방향 / 봉 굵기 / 윗꼬리 유무 / 윗꼬리 굵기 / 아랫꼬리 유무 / 아랫꼬리 굵기
len_df = len(df)

for i in range(1,len_df):
    df_raw1 = df.iloc[i]
    open1 = df_raw1['open']
    close1 = df_raw1['close']
    high1 = df_raw1['high']
    low1 = df_raw1['low']
    
    vec1 = [df_raw1['date'],0,0,0,0,0,0,0]  # target
    
    # vectorization
    temp = close1/open1
    if temp < 0.995 or temp > 1.005:
        vec1[1] = 1
    if temp >= 1:
        vec1[2] = 1
    temp = abs(close1-open1)
    if temp > open1*0.1:
        vec1[3] = 1
    close0 = df.iloc[i-1]['close']        
    lim_high1 = close0*1.3
    top1 = max(open1, close1)
    temp = high1 - top1
    if temp != 0:
        vec1[4] = 1
    if temp > 0.3*(lim_high1 - top1):
        vec1[5] = 1
    lim_low1 = close0*0.7
    bottom1 = min(open1, close1)
    temp = bottom1 - low1
    if temp != 0:
        vec1[6] = 1
    if temp > 0.3*(bottom1 - lim_low1):
        vec1[7] = 1
        
    df_res = df_res.append(pd.DataFrame([vec1], columns=['date','ce','cd','ct','he','ht','te','tt']))

In [80]:
df_res = df_res.reset_index(drop=True)
df_res.to_csv("vec/000020.csv")
df_res = pd.read_csv("vec/000020.csv", parse_dates=['date'], index_col=[0])
df_res

,date,ce,cd,ct,he,ht,te,tt
0,2012-01-03,1,1,0,1,0,0,0
1,2012-01-04,0,1,0,1,0,1,0
2,2012-01-05,1,1,0,1,0,1,0
3,2012-01-06,1,0,0,1,0,1,0
4,2012-01-09,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...
2169,2020-10-28,1,1,0,1,0,0,0
2170,2020-10-29,1,1,0,1,0,1,0
2171,2020-10-30,1,0,0,0,0,1,0
2172,2020-11-02,1,1,0,1,0,1,0


In [89]:
list(df_res.iloc[0][[1,2,3,4,5,6,7]])

[1, 1, 0, 1, 0, 0, 0]

In [91]:
N = 5
LEN = len(df_res)

In [93]:
df_res.head(20)

,date,ce,cd,ct,he,ht,te,tt
0,2012-01-03,1,1,0,1,0,0,0
1,2012-01-04,0,1,0,1,0,1,0
2,2012-01-05,1,1,0,1,0,1,0
3,2012-01-06,1,0,0,1,0,1,0
4,2012-01-09,1,1,0,1,0,1,0
5,2012-01-10,0,0,0,1,0,1,0
6,2012-01-11,1,1,0,1,1,0,0
7,2012-01-12,1,0,0,0,0,1,0
8,2012-01-13,1,0,0,1,0,0,0
9,2012-01-16,0,1,0,1,0,1,0


In [97]:
np_x = []
for i in range(N,LEN):
    elem = []
    for j in range(1,N+1):
        elem.append(list(df_res.iloc[i-j][[1,2,3,4,5,6,7]]))
    np_x.append(elem)
np_x = np.array(np_x)
np_x

array([[[1, 1, 0, ..., 0, 1, 0],
        [1, 0, 0, ..., 0, 1, 0],
        [1, 1, 0, ..., 0, 1, 0],
        [0, 1, 0, ..., 0, 1, 0],
        [1, 1, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 1, 0],
        [1, 1, 0, ..., 0, 1, 0],
        [1, 0, 0, ..., 0, 1, 0],
        [1, 1, 0, ..., 0, 1, 0],
        [0, 1, 0, ..., 0, 1, 0]],

       [[1, 1, 0, ..., 1, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        [1, 1, 0, ..., 0, 1, 0],
        [1, 0, 0, ..., 0, 1, 0],
        [1, 1, 0, ..., 0, 1, 0]],

       ...,

       [[1, 1, 0, ..., 0, 1, 0],
        [1, 1, 0, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 1, 0],
        [1, 0, 1, ..., 0, 1, 0],
        [1, 1, 0, ..., 0, 1, 0]],

       [[1, 0, 0, ..., 0, 1, 0],
        [1, 1, 0, ..., 0, 1, 0],
        [1, 1, 0, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 1, 0],
        [1, 0, 1, ..., 0, 1, 0]],

       [[1, 1, 0, ..., 0, 1, 0],
        [1, 0, 0, ..., 0, 1, 0],
        [1, 1, 0, ..., 0, 1, 0],
        [1, 1, 0, ..., 0, 0, 0],
        [1, 1, 0, ..

In [98]:
np_y = [list(df_res.iloc[i][[1,2,3,4,5,6,7]]) for i in range(N,LEN)]
np_y

[[0, 0, 0, 1, 0, 1, 0],
 [1, 1, 0, 1, 1, 0, 0],
 [1, 0, 0, 0, 0, 1, 0],
 [1, 0, 0, 1, 0, 0, 0],
 [0, 1, 0, 1, 0, 1, 0],
 [1, 1, 0, 1, 0, 1, 0],
 [1, 0, 0, 1, 0, 1, 0],
 [1, 0, 0, 0, 0, 1, 0],
 [1, 1, 0, 1, 0, 1, 0],
 [1, 1, 0, 1, 0, 1, 0],
 [1, 1, 0, 1, 0, 0, 0],
 [1, 0, 0, 1, 0, 1, 0],
 [1, 1, 0, 1, 0, 0, 0],
 [1, 0, 0, 1, 0, 1, 0],
 [0, 0, 0, 1, 0, 1, 0],
 [1, 0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 1, 0],
 [1, 0, 0, 1, 0, 1, 0],
 [0, 1, 0, 1, 0, 1, 0],
 [1, 1, 0, 1, 0, 1, 0],
 [1, 0, 0, 1, 0, 1, 0],
 [1, 1, 0, 0, 0, 1, 0],
 [1, 0, 0, 1, 0, 1, 0],
 [1, 0, 0, 1, 0, 1, 0],
 [1, 0, 0, 0, 0, 1, 0],
 [1, 1, 0, 1, 0, 0, 0],
 [1, 0, 0, 1, 0, 1, 0],
 [0, 0, 0, 1, 0, 1, 0],
 [1, 1, 0, 0, 0, 1, 0],
 [0, 1, 0, 1, 0, 1, 0],
 [0, 0, 0, 1, 0, 1, 0],
 [1, 0, 0, 1, 0, 0, 0],
 [1, 0, 0, 1, 0, 1, 0],
 [0, 1, 0, 1, 0, 1, 0],
 [1, 1, 0, 1, 0, 1, 0],
 [1, 0, 0, 0, 0, 1, 0],
 [1, 1, 0, 0, 0, 1, 0],
 [1, 0, 0, 1, 0, 0, 0],
 [1, 1, 0, 1, 0, 1, 0],
 [1, 1, 0, 1, 0, 0, 0],
 [1, 1, 0, 1, 0,

In [119]:
len(temp_vnt)

650

In [102]:
network = models.Sequential()

1518